In [ ]:
import numpy as np
from ALC import *
import pandas as pd 



#    [1,0] -> 0   (clase 0)
#    [0,1] -> 1   (clase 1)
def obtener_clases_reales(Yv):
    n = Yv.shape[1]
    y_true = np.zeros(n, dtype=int)

    for i in range(n):
        if Yv[0, i] == 1:
            y_true[i] = 0
        else:
            y_true[i] = 1

    return y_true



#Predicción de clases con un W dado
#    Para cada columna de Y_pred = W . Xv:
#    si y0 >= y1 -> clase 0
#    si y0 <  y1 -> clase 1
def predecir_todas(W, Xv):
    Y_pred = W @ Xv          
    n = Y_pred.shape[1]
    y_pred = np.zeros(n, dtype=int)

    for i in range(n):
        y0 = Y_pred[0, i]
        y1 = Y_pred[1, i]

        if y0 >= y1:
            y_pred[i] = 0
        else:
            y_pred[i] = 1

    return y_pred


# Matriz de confusión 2x2 
#    Filas = clase real, Columnas = clase predicha
#       [ [ reales 0 predichos 0, reales 0 predichos 1 ],
#         [ reales 1 predichos 0, reales 1 predichos 1 ] ]

def matriz_confusion(y_true, y_pred):
    matriz = np.zeros((2, 2), dtype=int)

    for t, p in zip(y_true, y_pred):
        matriz[int(t), int(p)] += 1

    return matriz



# Accuracy: cantidad de aciertos / total

def accuracy(y_true, y_pred):
    aciertos = 0
    total = len(y_true)

    for t, p in zip(y_true, y_pred):
        if t == p:
            aciertos += 1

    return aciertos / total


X_t,Y_t,Xv,Yv= cargarDataset("")

ranX = rango(X_t)
n,p = X_t.shape
if ranX == p and n>p:
    L,Lt=cholesky(traspuesta(X_t),X_t)
elif ranX == n:
    if n < p:
        L,Lt=cholesky(X_t,traspuesta(X_t))
    elif n == p:
        L,Lt=cholesky(X_t)

Qhh,Rhh= QR_con_HH(traspuesta(X_t))
Qgs,Rgs= QR_con_GS(traspuesta(X_t))
U,S,V= svd_reducida(X_t)

W_svd=pinvSVD(U,S,V,Y_t)
W_cholesky= pinvEcuacionesNormales(X_t,L,Y_t)
W_gs= pinvGramSchmidt(Qgs,Rgs,Y_t)
W_hh= pinvHouseHolder(Qhh,Rhh,Y_t)

# Clases reales a partir de Yv (one-hot)
y_real = obtener_clases_reales(Yv)

# Clases predichas usando W_svd
y_pred_svd = predecir_todas(W_svd, Xv)

# Matriz de confusión y accuracy para SVD
matriz_svd = matriz_confusion(y_real, y_pred_svd)
acc_svd = accuracy(y_real, y_pred_svd)

print("Matriz de confusión (SVD):")
print(matriz_svd)
print("Accuracy (SVD):", acc_svd)

y_pred_gs = predecir_todas(W_gs, Xv)

# Matriz de confusión y accuracy para SVD
matriz_gs = matriz_confusion(y_real, y_pred_gs)
acc_gs = accuracy(y_real, y_pred_gs)

print("Matriz de confusión (gs):")
print(matriz_gs)
print("Accuracy (gs):", acc_gs)

y_pred_cholesky = predecir_todas(W_cholesky, Xv)

# Matriz de confusión y accuracy para cholesky
matriz_cholesky = matriz_confusion(y_real, y_pred_cholesky)
acc_cholesky = accuracy(y_real, y_pred_cholesky)

print("Matriz de confusión (cholesky):")
print(matriz_cholesky)
print("Accuracy (cholesky):", acc_cholesky)

y_pred_hh = predecir_todas(W_hh, Xv)

# Matriz de confusión y accuracy para cholesky
matriz_hh = matriz_confusion(y_real, y_pred_hh)
acc_hh = accuracy(y_real, y_pred_hh)

print("Matriz de confusión (householder):")
print(matriz_hh)
print("Accuracy (householder):", acc_hh)




Matriz de confusión (SVD):
[[334 166]
 [150 350]]
Accuracy (SVD): 0.684
Matriz de confusión (gs):
[[334 166]
 [150 350]]
Accuracy (gs): 0.684
Matriz de confusión (cholesky):
[[334 166]
 [150 350]]
Accuracy (cholesky): 0.684
Matriz de confusión (householder):
[[334 166]
 [150 350]]
Accuracy (householder): 0.684


In [9]:


# Datos
metodos = ["Gram-Schmidt", "Householder", "SVD", "Cholesky"]
tiempos = [11, 28, 55, 56]  # en minutos
accuracy = [0.684, 0.684, 0.684, 0.684]  # mismo valor para todos

# Crear DataFrame
tabla = pd.DataFrame({
    "Método": metodos,
    "Tiempo (minutos)": tiempos,
    "Accuracy": accuracy
})

print(tabla)


         Método  Tiempo (minutos)  Accuracy
0  Gram-Schmidt                11     0.684
1   Householder                28     0.684
2           SVD                55     0.684
3      Cholesky                56     0.684


En este trabajo aplicamos distintos metodos para obtener la matriz de pesos W, lo que nos permite aproximar la relacion entre los embeddings de entrada X y las etiquetas Y calculando la pseudoinversa. 
Utilizamos 4 metodos: 

"Ecuaciones normales con Cholesky"
"Descomposicion QR usando Gram-Schmidt
"Descomposicion QR usando House-Holder
"Descomposicion en Valores singulares.

Primero implementamos las funciones auxiliares manualmente  (producto externo, traspuesta, norma, multiplicación), respetando las restricciones del laboratorio.  Las funciones del labo andaban bien para matrices pequeñas, pero cuando intentamos aplicarlas sobre los embeddings reales de tamaño 1536×2000 y 2000×2, se volvieron extremadamente lentas e incluso nuestro código llegó a sobrecalentar la computadora. Esto se debe a que las versiones originales realizaban bucles anidados, lo cual funcionaba para los casos de test con matrices pequeñas, pero el costo computacional se hacía inviable para matrices tan grandes. Por eso, mantuvimos la estructura de nuestros algoritmos, pero cambiamos algunas partes internas usando funciones de NumPy (como @ para multiplicación, slicing y np.sum), logrando una gran mejora en tiempos (el código terminó de correr sin explotar la memoria, y en menos de una hora).

Aplicamos los cuatro metodos para obtener la matriz W. La guardamos para cada caso y luego la usamos para validar sobre Xv e Yv. Todos los métodos predijeron exactamente lo mismo, tuvieron la misma matriz de confusion. Esto dio una accuracy del 68.4% en los cuatro casos. Esto nos indica que, al menos para este problema, los métodos convergen a soluciones equivalentes y que el sistema no está mal condicionado.

La gran diferencia estuvo en los tiempos: Gram-Schmidt fue el más rápido, tardando 11 minutos. Householder tardó 28 minutos, SVD tardó 55 minutos, y Cholesky fue la mas lenta, tardando 56 minutos. 

En conclusión, aunque todos los métodos dieron el mismo resultado de clasificación, no todos son igual de prácticos si tenemos en cuenta los tiempos de ejecución. Además, comprobamos que fue necesario optimizar las funciones del laboratorio para que el cálculo sea viable en matrices reales de gran tamaño.
.



